In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert import download
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import os
from transformers import BertModel
from zipfile import ZipFile

In [6]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [7]:
def get_pytorch_kobert_model(ctx="cpu", cachedir=".cache"):
    
    def get_kobert_model(model_path, vocab_file, ctx="cpu"):
        bertmodel = BertModel.from_pretrained(model_path)
        device = torch.device(ctx)
        bertmodel.to(device)
        bertmodel.eval()
        vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
            vocab_file, padding_token="[PAD]"
        )
        return bertmodel, vocab_b_obj

    pytorch_kobert = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/models/kobert_v1.zip",
        "chksum": "411b242919",  # 411b2429199bc04558576acdcac6d498
    }
    
    # download model
    model_info = pytorch_kobert
    model_path, is_cached = download(
        model_info["url"], model_info["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    zipf = ZipFile(os.path.expanduser(model_path))
    zipf.extractall(path=cachedir_full)
    model_path = os.path.join(os.path.expanduser(cachedir), "kobert_from_pretrained")
    # download vocab
    vocab_path = get_tokenizer()
    return get_kobert_model(model_path, vocab_path, ctx)

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_v1.zip
using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
df = pd.read_csv(r'C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\input_data.csv',index_col=0)
df

,제목,상승하락
0,"대한생명, 원금 100% 환급해주는 변액보험 출시",0
1,"코스피, 새해 첫날 혼조세",0
2,[2010 경제대전망]짝수해 징크스는 없다..코스피 2120찍는다,0
3,"<생생코스닥>주간추천주 결산-한화證, 575% 수익률 기염",0
4,"[타이쿤의 시장분석] 전자주, 2차 상승 국면이다.",0
5,“올 증시 20%수익 저평가 종목 공략”<펀드매니저 설문>,0
6,[투자전략]조정 우려 주식비중 줄여야-김병연,0
7,"[초점]""1월 효과 있다 1750까지 가능""",0
8,"코스피, 1690 탈환..1691.10",0
9,[초점]'1월 효과' 있다 1750까지 가능,0


In [10]:
encoder = LabelEncoder()
encoder.fit(df['상승하락'])
df['상승하락'] = encoder.transform(df['상승하락'])
df

,제목,상승하락
0,"대한생명, 원금 100% 환급해주는 변액보험 출시",0
1,"코스피, 새해 첫날 혼조세",0
2,[2010 경제대전망]짝수해 징크스는 없다..코스피 2120찍는다,0
3,"<생생코스닥>주간추천주 결산-한화證, 575% 수익률 기염",0
4,"[타이쿤의 시장분석] 전자주, 2차 상승 국면이다.",0
5,“올 증시 20%수익 저평가 종목 공략”<펀드매니저 설문>,0
6,[투자전략]조정 우려 주식비중 줄여야-김병연,0
7,"[초점]""1월 효과 있다 1750까지 가능""",0
8,"코스피, 1690 탈환..1691.10",0
9,[초점]'1월 효과' 있다 1750까지 가능,0


In [11]:
df['제목'] = df['제목'].astype(str)
df['상승하락'] = df['상승하락'].astype(str)

In [12]:
data = df.values.tolist()
data

[['대한생명, 원금 100% 환급해주는 변액보험 출시', '0'],
 ['코스피, 새해 첫날 혼조세', '0'],
 ['[2010 경제대전망]짝수해 징크스는 없다..코스피 2120찍는다', '0'],
 ['<생생코스닥>주간추천주 결산-한화證, 575% 수익률 기염', '0'],
 ['[타이쿤의 시장분석] 전자주, 2차 상승 국면이다.', '0'],
 ['“올 증시 20%수익 저평가 종목 공략”<펀드매니저 설문>', '0'],
 ['[투자전략]조정 우려 주식비중 줄여야-김병연', '0'],
 ['[초점]"1월 효과 있다 1750까지 가능"', '0'],
 ['코스피, 1690 탈환..1691.10', '0'],
 ["[초점]'1월 효과' 있다 1750까지 가능", '0'],
 ["NH-CA운용 '레버리지 인덱스펀드', 금감원 선정 금융신상품 최우수상", '0'],
 ['작년 파생상품 거래 사상 최고', '0'],
 ['반도체주 “해피 뉴 이어”', '0'],
 ['코스피, 오름폭 확대 1690선 탈환', '0'],
 ['추가 상승이냐 vs 게걸음 장세냐', '0'],
 ['[생생코스닥]우리투자ㆍ대신證 ‘탁월한 안목’ 빛났다', '0'],
 ["대주주가 가장 맛있어 하는 '유상감자'", '0'],
 ["코스피 소폭 상승 외국인 기관 '사자'", '0'],
 ['올 첫 증시 큰폭 상승 虎 虎 虎 호쾌한 스타트', '0'],
 ['코스피, 경인년 기대감 매수세 유입 - 동시호가', '0'],
 ['한화證, 코스피200 스텝다운 ELS 등 2종 판매', '0'],
 ['LED 2차전지 등 녹색업종 올해도 ‘강추’', '0'],
 ['"올 증시 上低下高 1,900까지 오를것"', '0'],
 ["2010년 '白호랑이 해' 증시 개막", '0'],
 ['코스닥 520선 재돌파 코스피는 강보합', '0'],
 ['[증권가 소식] 한국투자증권 外', '0'],
 ["코스피 1690선 돌파 외국인 기관 '사자'", '0'],
 ['한화證, 코스피200스텝다운 ELS 2종 판매

In [13]:
train, test = train_test_split(data, test_size = 0.2, shuffle = False, random_state=777)
print(len(train))
print(len(test))

173617
43405


In [14]:
tokenizer = get_tokenizer()

using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [30]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 8 #64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


In [31]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [32]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size) # num_workers=5
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size) #num_workers=5

In [33]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [34]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [35]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [36]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 0.683738648891449 train acc 0.625
epoch 1 batch id 201 loss 0.6717503666877747 train acc 0.5074626865671642
epoch 1 batch id 401 loss 0.7331762313842773 train acc 0.4884663341645885
epoch 1 batch id 601 loss 1.0271552801132202 train acc 0.512063227953411
epoch 1 batch id 801 loss 0.7574307322502136 train acc 0.49001248439450684
epoch 1 batch id 1001 loss 0.7958363890647888 train acc 0.49512987012987014
epoch 1 batch id 1201 loss 0.7617783546447754 train acc 0.47991257285595335
epoch 1 batch id 1401 loss 0.41026875376701355 train acc 0.4868843683083512
epoch 1 batch id 1601 loss 0.48480555415153503 train acc 0.47891942535915055
epoch 1 batch id 1801 loss 0.30509594082832336 train acc 0.4809827873403665
epoch 1 batch id 2001 loss 0.3447790741920471 train acc 0.48282108945527236
epoch 1 batch id 2201 loss 2.5869007110595703 train acc 0.5015901862789641
epoch 1 batch id 2401 loss 0.060073308646678925 train acc 0.5077051228654728
epoch 1 batch id 2601 loss 0.03458105

epoch 1 test acc 0.5469959454478437


epoch 2 batch id 1 loss 1.6566886901855469 train acc 0.0
epoch 2 batch id 201 loss 1.0089751482009888 train acc 0.40733830845771146
epoch 2 batch id 401 loss 1.7438609600067139 train acc 0.5152743142144638
epoch 2 batch id 601 loss 3.177328586578369 train acc 0.5447171381031614
epoch 2 batch id 801 loss 0.29154080152511597 train acc 0.5051498127340824
epoch 2 batch id 1001 loss 0.8612172603607178 train acc 0.477022977022977
epoch 2 batch id 1201 loss 0.8946592807769775 train acc 0.4650291423813489
epoch 2 batch id 1401 loss 0.2625780403614044 train acc 0.48643825838686655
epoch 2 batch id 1601 loss 0.7263746857643127 train acc 0.47009681449094315
epoch 2 batch id 1801 loss 0.22431397438049316 train acc 0.49181010549694615
epoch 2 batch id 2001 loss 0.2129485309123993 train acc 0.4926911544227886
epoch 2 batch id 2201 loss 3.1601061820983887 train acc 0.5105633802816901
epoch 2 batch id 2401 loss 0.03661824017763138 train acc 0.5159308621407747
epoch 2 batch id 2601 loss 0.0253273658454

epoch 2 test acc 0.5469959454478437


epoch 3 batch id 1 loss 1.7580115795135498 train acc 0.0
epoch 3 batch id 201 loss 1.1886080503463745 train acc 0.40733830845771146
epoch 3 batch id 401 loss 1.8222267627716064 train acc 0.516209476309227
epoch 3 batch id 601 loss 3.089641809463501 train acc 0.5453410981697171
epoch 3 batch id 801 loss 0.25662150979042053 train acc 0.5056179775280899
epoch 3 batch id 1001 loss 0.4178735315799713 train acc 0.4807692307692308
epoch 3 batch id 1201 loss 1.2126739025115967 train acc 0.4855328892589509
epoch 3 batch id 1401 loss 0.23740455508232117 train acc 0.5041934332619558
epoch 3 batch id 1601 loss 1.0349889993667603 train acc 0.48852279825109307
epoch 3 batch id 1801 loss 0.1929190456867218 train acc 0.5097168239866741
epoch 3 batch id 2001 loss 0.2163906991481781 train acc 0.508808095952024
epoch 3 batch id 2201 loss 3.1587600708007812 train acc 0.5252158109950023
epoch 3 batch id 2401 loss 0.0381704606115818 train acc 0.529362765514369
epoch 3 batch id 2601 loss 0.02907431684434414 

epoch 3 test acc 0.5469959454478437


epoch 4 batch id 1 loss 1.800036907196045 train acc 0.0
epoch 4 batch id 201 loss 1.4602949619293213 train acc 0.40733830845771146
epoch 4 batch id 401 loss 1.7290483713150024 train acc 0.516209476309227
epoch 4 batch id 601 loss 2.3509654998779297 train acc 0.5453410981697171
epoch 4 batch id 801 loss 0.21518555283546448 train acc 0.5056179775280899
epoch 4 batch id 1001 loss 0.2641701102256775 train acc 0.492007992007992
epoch 4 batch id 1201 loss 1.4887816905975342 train acc 0.4949000832639467
epoch 4 batch id 1401 loss 0.19622263312339783 train acc 0.5122234118486795
epoch 4 batch id 1601 loss 1.3681365251541138 train acc 0.49554965646470955
epoch 4 batch id 1801 loss 0.1847558170557022 train acc 0.5159633536923931
epoch 4 batch id 2001 loss 0.1941184103488922 train acc 0.5144302848575713
epoch 4 batch id 2201 loss 2.461129903793335 train acc 0.5303271240345298
epoch 4 batch id 2401 loss 0.06946107000112534 train acc 0.5340483132028322
epoch 4 batch id 2601 loss 0.04225786402821541

epoch 4 test acc 0.5469959454478437


epoch 5 batch id 1 loss 1.7924410104751587 train acc 0.0
epoch 5 batch id 201 loss 1.7256516218185425 train acc 0.40733830845771146
epoch 5 batch id 401 loss 1.891640543937683 train acc 0.516209476309227
epoch 5 batch id 601 loss 2.018383026123047 train acc 0.5453410981697171
epoch 5 batch id 801 loss 0.17479519546031952 train acc 0.5056179775280899
epoch 5 batch id 1001 loss 0.20388583838939667 train acc 0.492007992007992
epoch 5 batch id 1201 loss 1.7593828439712524 train acc 0.4949000832639467
epoch 5 batch id 1401 loss 0.1855909675359726 train acc 0.5122234118486795
epoch 5 batch id 1601 loss 1.7110111713409424 train acc 0.49554965646470955
epoch 5 batch id 1801 loss 0.17803330719470978 train acc 0.5159633536923931
epoch 5 batch id 2001 loss 0.16423861682415009 train acc 0.5144302848575713
epoch 5 batch id 2201 loss 2.0390849113464355 train acc 0.5303271240345298
epoch 5 batch id 2401 loss 0.1466907113790512 train acc 0.5340483132028322
epoch 5 batch id 2601 loss 0.1161810830235481

epoch 5 test acc 0.5469959454478437
